# Auswahl und Training eines Modells
Bis hierhin gekommen, haben Sie das Analyseproblem erfasst, Sie haben die Daten erhalten und untersucht, Sie haben ein Trainingsset und ein Testset erstellt und Sie haben Transformationspipelines geschrieben, um Ihre Daten zu bereinigen und automatisch für die Algorithmen des maschinellen Lernens vorzubereiten. Sie sind nun bereit, ein Modell für maschinelles Lernen auszuwählen und zu trainieren. Ab jetzt wird es auch einfacher werden, als Sie denken. Der größte Aufwand besteht bei maschinellem Lernen immer in der Vorbereitung der Daten. Zunächst bereiten wir wieder unsere Daten vor, wie in Kapitel 4 gezeigt.

In [3]:
#Datenvorbereitung aus Kapitel 4
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator , TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# Datenbeschaffung / Einlesen einer csv Datei wie in Abschnitt 2 beschrieben
def load_housing_data():
    csv_path = os.path.join("datasets/housing/housing.csv")
    return pd.read_csv(csv_path)

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room

    def fit(self, X, y=None):
        return self # nothing else to do

    # Da die Methode nur die Werte übergeben bekommt. muss auf das Array mit Spaltennummern (rooms_ix etc.) zugegriffen werden
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

housing = load_housing_data()

# Erstellung income category Attribut mit fünf Kategorien
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

# Basierend auf dem Kategorie-Attribut wird nun eine stratifizierte Stichprobe gezogen
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
housing = strat_train_set.drop("median_house_value",axis=1)

housing_labels = strat_train_set["median_house_value"].copy()

# Erstellen eines Dataframes ohne kategorielle Attribute
housing_num = housing.drop("ocean_proximity", axis=1)

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

# Klasse für die Auswahl nummerischer und kategorieller Spalten
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

# Pipeline für ide Verarbeitung nummerischer Attribute
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
    ])

# Pipeline für die Verarbeitung kategorieller Attribute
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('label_binarizer', OneHotEncoder()),
    ])

# Zusammensetzen der Teil-Pipelines
full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
    ])

# Bis hierher arbeitet die pipeline noch nicht mit echten Daten. Sie verfügt nur über das Wissen über die Attribute und der 
# Transformationsfunktionen. Erst jetzt werden der Pipeline echte housing-Daten übergeben:
housing_prepared = full_pipeline.fit_transform(housing)



Lassen Sie uns nun mit einer einfachen linearen regression starten. Dafür bietet sklearn die Klasse LinearRegression an:

In [29]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Das wars! Sie haben Ihre erste Datenanalyse mit Python erstellt. Lassen Sie uns das Modell an ein par Testdaten aus dem Trainingsdatensatz ausprobieren.

In [30]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:\t", lin_reg.predict(some_data_prepared))

print("Labels:\t\t", list(some_labels))

Predictions:	 [203682.37421606 326371.39500521 204218.64541583  58685.47562356
 194213.06476433]
Labels:		 [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


Es funktioniert. Auch wenn die Vorhersagen nicht genau und präzise sind, so stimmt die grundsätzliche Richtung. Lassen Sie uns nun die Leistungsfähigkeit des Modells messen. Dafür haben wir in Kapitel 1 das Root Mean Square Error (RMSE) Verfahren kennengelernt. Wendne Wir nun das RMSE-Verfahren auf dem ganzen Trainingsdatensatz an. Sklearn stellt dafür die Klasse **mean_squared_error** zur Verfügung.

In [31]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68376.6429545994

Okay, das ist besser als nichts, aber eindeutig kein tolles Ergebnis: Die Medianwerte der meisten Distrikte liegen zwischen 120.000 und 265.000 Dollar, so dass ein typischer Prognosefehler von 68.628 Dollar nicht sehr zufriedenstellend ist. Dies ist ein Beispiel für ein Modell, für das die Trainingsdaten nicht ausreichen. Wenn dies geschieht, kann es bedeuten, dass die Merkmale nicht genügend Informationen für gute Vorhersagen liefern oder dass das Modell nicht leistungsfähig genug ist. Wie wir im vorigen Kapitel gesehen haben, bestehen die Hauptmöglichkeiten zur Behebung der Ungenauigkeit darin, die Einschränkungen des Modells zu reduzieren. 

Sie könnten versuchen, weitere Attribute hinzuzufügen (z.B. das Logbuch der Bevölkerung), aber lassen Sie uns zuerst ein komplexeres Modell ausprobieren, um zu sehen, wie es funktioniert. Lassen Sie uns einen **Entscheidungsbaum-Regressor** trainieren. Dies ist ein leistungsfähiges Modell, das in der Lage ist, komplexe nichtlineare Beziehungen in den Daten zu finden. Der Code sollte inzwischen bekannt aussehen:

In [32]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

Jetzt ist das Modell trainiert. Werten wir es nun aus:

In [33]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

Was? Überhaupt kein Fehler? Könnte dieses Modell wirklich absolut perfekt sein? Natürlich ist es viel wahrscheinlicher, dass das Modell die Daten schlecht ausgewertet hat. Wie können Sie sicher sein?, dass es wirklich ein schlechtes Modell ist? Wie wir in Kapitel 4 gesehen haben, wollen Sie den Testsatz nicht anfassen, bevor Sie nicht bereit sind, ein Modell zu starten, von dem Sie überzeugt sind. Also müssen Sie einen Teil des Trainingssatzes für das Training und einen Teil für die Modellvalidierung verwenden.

Dies kann durch die Verwendung der Scikit-Learn-Funktion zur Kreuzvalidierung geschehen. Der folgende Code führt eine **K-Fold-Kreuzvalidierung** durch: Er teilt den Trainingssatz nach dem Zufallsprinzip in 10 verschiedene Teilmengen, die als *Folds* bezeichnet werden, dann trainiert und bewertet er das Entscheidungsbaum-Modell 10 Mal, wobei er jedes Mal einen anderen *Fold* zur Bewertung auswählt und auf den anderen 9 Folds trainiert. Das Ergebnis ist ein Array mit den 10 Bewertungsergebnissen. 


In [34]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

Schauen wir uns die Ergebnisse an.

In [35]:
# Funktion für die Rückgabe des Qualitätsmerkmals  Scores
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

Scores: [68662.81444455 66566.25492662 71306.45872488 69075.20867824
 71104.76073849 75332.84866504 71612.50623391 71311.06830579
 77272.0399851  70674.06173656]
Mean: 71291.80224391731
Standard deviation: 2945.65758114086


Jetzt sieht der Entscheidungsbaum nicht mehr so gut aus wie vorher. Tatsächlich scheint er schlechter als das Modell der linearen Regression zu sein! Beachten Sie, dass die Kreuzvalidierung Ihnen nicht nur eine Schätzung der Leistung Ihres Modells ermöglicht, sondern auch ein Maß dafür, wie genau diese Schätzung ist (d.h. ihre Standardabweichung). Der Entscheidungsbaum hat einen Wert von ungefähr 71.200, ± 3.000. Diese Informationen lägen Ihnen nicht vor, wenn Sie nur einen Validierungssatz verwendet hätten. Aber die Kreuzvalidierung geht auf Kosten des Modelltrainings, was nicht immer möglich ist.
**Aufgabe 1:** Berechnen Sie die gleichen Scores für das lineare Regressionsmodell.

In [36]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [66877.52329047 66608.12020249 70575.91162015 74179.94793032
 67683.3220154  71103.16844079 64782.65900243 67711.2994259
 71080.40466615 67687.63849832]
Mean: 68828.99950924338
Standard deviation: 2662.7615667978857


Der Entscheidungsbaum-Algorithmus ist offenbar noch schlechter als die lineare Regression. Lassen Sie uns noch ein letztes Modell ausprobieren: den **RandomForestRegressor**. RandomForest arbeitet, indem es viele Entscheidungsbäume auf zufällige Teilmengen der Attribute trainieren und dann den Druchschnitt ihrer Ergebnisse zu ermitteln. Der Aufbau eines Modells auf den Ergebnissen anderer Modelle wird als *Ensemble-Learning* bezeichnet und es ist oft eine gute Möglichkeit, ML-Algorithmen noch weiter voranzutreiben.
**Aufgabe 2:** Erstellen Sie einen randomForestRegressor und berechnen Sie die Scores für dessen Performance. Orientieren Sie sich dabei an den oberen Beispielen.

In [41]:
from sklearn.ensemble import RandomForestRegressor
# ToDo